In [ ]:
import sklearn
import os
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import roc_auc_score

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sample_submission=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
from pandas_profiling import ProfileReport

ProfileReport(train)

In [ ]:
train_ds = train.drop(["id","target"],axis=1)
labels = train["target"]
test_ds = test.drop(["id"],axis=1)

print("--------DETAILS OF TRAINING AND TESTING DATA SIZE---------")
print("Number of instances in train data: ",len(train_ds))
print("Number of instances in test data: ",len(test_ds))
print("There are {} feature columns".format(test_ds.shape[1]))

cat_cols = []
num_cols = []

for col in test_ds.columns:
    if test_ds[col].dtype == "float64":
        num_cols.append(col)
    else:
        cat_cols.append(col)
        
print("\n--------DETAILS OF COLUMNS-------")
print("There are {} categorical and {} numerical columns.".format(len(cat_cols),len(num_cols)))
print("Categorical columns are: ",cat_cols)
print("Numerical columns are: ",num_cols)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(train.corr())
plt.show()

In [ ]:
cat_encoder = LabelEncoder()
scaler = StandardScaler()


ds = pd.concat([train_ds,test_ds],axis=0)
for col in cat_cols:
    ds[col] = cat_encoder.fit_transform(ds[col])

ds[cat_cols] = scaler.fit_transform(ds[cat_cols])

train_ds = ds.iloc[:len(train_ds),:]
test_ds = ds.iloc[len(train_ds):,:]

train_ds.head()

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold,GridSearchCV

In [ ]:
SEED = 1024
N_SPLITS =5
kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

In [ ]:
# model = lgb.LGBMClassifier()
# param = {'n_estimators':[150,200,225,250,275,300,400,450,500,550,600,650,700,750,800,850,900,1000]}
# clf=GridSearchCV(model,param,scoring='roc_auc',refit=True,cv=10)
# clf.fit(train_ds,labels)
# print('Best roc_auc: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_))

In [ ]:
i=1
pred_test_full =0
max_auc = 0
lgbmodel=lgb.LGBMClassifier(n_estimators=1000)
for train_idx,test_idx in kfold.split(train_ds,labels):
    print(' Running {} of KFold {}'.format(i,kfold.n_splits))
    xtr,xvl = train_ds.loc[train_idx],train_ds.loc[test_idx]
    ytr,yvl = labels.loc[train_idx],labels.loc[test_idx]
    lgbmodel.fit(xtr,ytr)
    score = roc_auc_score(yvl,lgbmodel.predict(xvl))   
    print('ROC AUC score:',score)
        
    pred_test = lgbmodel.predict_proba(test_ds)[:,1]
    pred_test_full +=pred_test
    i+=1

In [ ]:

y_pred = pred_test_full/N_SPLITS

pred_csv = pd.read_csv("../input/tabular-playground-series-mar-2021/sample_submission.csv")
pred_csv.target = y_pred

pred_csv.to_csv("submission.csv",index=False)